In [1]:
import pandas as pd
import numpy as np
import os
import regex as re
import string

In [53]:
class Node():
    def __init__(self):
       # Note that using dictionary for children (as in this implementation) would not allow lexicographic sorting mentioned in the next section (Sorting),
       # because ordinary dictionary would not preserve the order of the keys
        self.children = {}  # mapping from character ==> Node
        self.value = None

def find(node, key):
    for char in key:
        if char in node.children:
            node = node.children[char]
        else:
            return None
    return node.value
    
def find_multiple(node, keys):
    vals = [None]*len(keys)
    print (vals)
    counter = 0
    for key in keys:
        for char in key:
            if char in node.children:
                node = node.children[char]
        print (node.value)
        vals[counter] = node.value
        counter += 1
    return vals

def update(node, key, difference):
    for char in key:
        if char in node.children:
            node = node.children[char]
    node.value += difference
    
def insert(root, string, value):
    node = root
    index_last_char = None
    for index_char, char in enumerate(string):
        if char in node.children:
            node = node.children[char]
        else:
            index_last_char = index_char
            break

    # append new nodes for the remaining characters, if any
    if index_last_char is not None: 
        for char in string[index_last_char:]:
            node.children[char] = Node()
            node = node.children[char]

    # store value in the terminal node
    node.value = value

    

In [44]:
listtopics=set()
listplaces=set()
cntnotop=0 
cntnoplc=0 
trieTopics = Node()
trieLoc = Node()
WordCount = Node()
topicTotalCount = Node()
topicAvgCount = Node()
placeAvgCount = Node()
placetotalCount = Node()

for i in range(0,22):
    if(i>=10):
        filename = 'reut2-0'+str(i)+'.sgm'
    else:
        filename = 'reut2-00'+str(i)+'.sgm'
    path = ''+filename
    file = open(path, 'rb')
    data = file.read()
#    x = re.findall(r'<REUTERS(.*?)</REUTERS>', data.decode("utf-8"), re.DOTALL, overlapped=True)
    x = re.findall(r'<REUTERS(.*?)</REUTERS>', data.decode("windows-1252"), re.DOTALL, overlapped=True)

    #topicCounter =0
    
    for j in range(0,len(x)):
        
        yTopic = re.findall(r'<TOPICS>(.*?)</TOPICS>', x[j], re.DOTALL, overlapped=True)
        for k in range(0,len(yTopic)):
            lt = yTopic[k]
            
            topics = re.findall(r'<D>(.*?)</D>', lt, re.DOTALL, overlapped=True)
            if(len(topics)==0):
                cntnotop=cntnotop+1
            for l in topics:
                if (find(trieTopics,l) == None):
                    insert(trieTopics, l, 1)
                else:
                    update(trieTopics, l, 1)
                listtopics.add(l)
                #topicCounter += 1
                  
        yPlace = re.findall(r'<PLACES>(.*?)</PLACES>', x[j], re.DOTALL, overlapped=True)
        for k in range(0,len(yPlace)):
            lt = yPlace[k]
            places = re.findall(r'<D>(.*?)</D>', lt, re.DOTALL, overlapped=True)
            if(len(places)==0):
                cntnoplc=cntnoplc+1
            for l in places:
                if (find(trieLoc, l) == None):
                    insert(trieLoc, l, 1)
                else:
                    update(trieLoc, l, 1)
                listplaces.add(l)

                    
        yBody = re.findall(r'<BODY>(.*?)</BODY>', x[j], re.DOTALL, overlapped=True)
        for b,word in enumerate(yBody):
            body = word.split()
            body = [element.lower() for element in body] ; body
            
           # insert(placeTotalCount, )
            
            for l in body:
                if (find(WordCount, l) == None):
                    insert(WordCount, l, 1)
                else:
                    update(WordCount, l, 1)
 
print(listtopics)
print(cntnotop)
print(listplaces)
print(cntnoplc)

{'red-bean', 'income', 'cocoa', 'corn', 'instal-debt', 'saudriyal', 'can', 'wool', 'palladium', 'stg', 'crude', 'nickel', 'pork-belly', 'peseta', 'cornglutenfeed', 'housing', 'zinc', 'lumber', 'trade', 'lit', 'hog', 'austdlr', 'veg-oil', 'meal-feed', 'wpi', 'coconut-oil', 'dkr', 'potato', 'cotton-oil', 'sunseed', 'carcass', 'soy-oil', 'silver', 'rupiah', 'copra-cake', 'money-fx', 'naphtha', 'palmkernel', 'cottonseed', 'palm-oil', 'rand', 'grain', 'nat-gas', 'rubber', 'sun-oil', 'sun-meal', 'corn-oil', 'lin-meal', 'sugar', 'earn', 'linseed', 'propane', 'coffee', 'barley', 'oat', 'skr', 'jobs', 'livestock', 'dfl', 'inventories', 'fuel', 'coconut', 'ringgit', 'ship', 'tea', 'dlr', 'copper', 'groundnut-oil', 'rice', 'gas', 'iron-steel', 'hk', 'sfr', 'strategic-metal', 'interest', 'nzdlr', 'rye', 'cpi', 'lei', 'rape-meal', 'f-cattle', 'soybean', 'tapioca', 'pet-chem', 'jet', 'castor-oil', 'retail', 'platinum', 'cruzado', 'reserves', 'acq', 'cotton', 'castorseed', 'gnp', 'lin-oil', 'alum', '

In [4]:
trieTopics = Node()
trieVal = 0
for x in listtopics:
    myTopic = listtopics.pop()
    listtopics.add(myTopic)
    insert(trieTopics, myTopic, trieVal)
    trieVal += 1
    
trieLoc = Node()
trieVal2 = 0
for x in listtopics:
    myLoc = listplaces.pop()
    listplaces.add(myLoc)
    insert(trieLoc, myLoc, trieVal2)
    trieVal2 += 1

In [5]:
for x in listtopics:
    print (listtopics.pop())
    listtopics.add(x)
    print (len(listtopics))

livestock
120
gold
120
yen
120
sunseed
120
trade
120
pork-belly
120
zinc
120
red-bean
120
alum
120
instal-debt
120
coconut-oil
120
nzdlr
120
rape-meal
120
veg-oil
120
sugar
120
bfr
120
orange
120
rice
120
palmkernel
120
plywood
120
coffee
120
lit
120
propane
120
earn
120
cornglutenfeed
120
l-cattle
120
cruzado
120
silver
120
nat-gas
120
corn-oil
120
lin-meal
120
groundnut-oil
120
copra-cake
120
money-fx
120
tea
120
wheat
120
strategic-metal
120
lead
120
ringgit
120
castorseed
120
dlr
120
barley
120
crude
120
soy-oil
120
jobs
120
carcass
120
lei
120
corn
120
linseed
120
f-cattle
120
dkr
120
bop
120
skr
120
acq
120
grain
120
retail
120
rapeseed
120
copper
120
ipi
120
cocoa
120
soybean
120
inventories
120
platinum
120
jet
120
palm-oil
120
austdlr
120
housing
120
castor-oil
120
dfl
120
groundnut
120
interest
120
fishmeal
120
stg
120
rand
120
nickel
120
meal-feed
120
gas
120
fuel
120
heat
120
naphtha
120
rape-oil
120
sun-meal
120
palladium
120
lin-oil
120
gnp
120
wool
120
sun-oil
120
dmk
12

In [47]:
#update(trieLoc, "west-germany", 4)
print (find(trieLoc, "usa"))

12542


In [46]:
print(find(WordCount, "sugar"))

693


In [35]:
vals = [None]*4
vals[2] = 3
print (vals)

[None, None, 3, None]


In [54]:
print (find_multiple(trieLoc, ['usa', 'west-germany']))

[None, None]
12542
12542
[12542, 12542]
